In [ ]:
## 설치
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,076 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,843 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restr

In [ ]:
## 초기화
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [ ]:
import pyspark
from pyspark.sql import DataFrame, SparkSession

In [ ]:
## Session 생성
spark = (
    SparkSession
    .builder
    .appName("Commerce Session")
    .getOrCreate()
)

spark

In [ ]:
%%time
df = spark.read.csv("/content/drive/MyDrive/order_products__prior.csv", header=True, inferSchema=True)
df.show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       2|     33120|                1|        1|
|       2|     28985|                2|        1|
|       2|      9327|                3|        0|
|       2|     45918|                4|        1|
|       2|     30035|                5|        0|
|       2|     17794|                6|        1|
|       2|     40141|                7|        1|
|       2|      1819|                8|        1|
|       2|     43668|                9|        0|
|       3|     33754|                1|        1|
|       3|     24838|                2|        1|
|       3|     17704|                3|        1|
|       3|     21903|                4|        1|
|       3|     17668|                5|        1|
|       3|     46667|                6|        1|
|       3|     17461|                7|        1|
|       3|     32665|                8|        1|


In [ ]:
%%time
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, ShortType

struct_schema = StructType([
    StructField("order_id", IntegerType()),
    StructField("product_id", IntegerType()),
    StructField("add_to_cart_order", ShortType()),
    StructField("reordered", ShortType())
])

df = spark.read.csv("/content/drive/MyDrive/order_products__prior.csv", header=True, schema=struct_schema)
df.show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       2|     33120|                1|        1|
|       2|     28985|                2|        1|
|       2|      9327|                3|        0|
|       2|     45918|                4|        1|
|       2|     30035|                5|        0|
|       2|     17794|                6|        1|
|       2|     40141|                7|        1|
|       2|      1819|                8|        1|
|       2|     43668|                9|        0|
|       3|     33754|                1|        1|
|       3|     24838|                2|        1|
|       3|     17704|                3|        1|
|       3|     21903|                4|        1|
|       3|     17668|                5|        1|
|       3|     46667|                6|        1|
|       3|     17461|                7|        1|
|       3|     32665|                8|        1|


In [ ]:
df1 = spark.read.csv("/content/drive/MyDrive/order_products__prior.csv", header=True, schema=struct_schema)
df2 = spark.read.csv("/content/drive/MyDrive/order_products__train.csv", header=True, schema=struct_schema)

df = df1.union(df2)
df = df.filter("order_id % 13 = 1")

In [ ]:
from pyspark.sql.types import StringType

struct_schema_product = StructType([
    StructField("product_id", IntegerType()),
    StructField("product_name", StringType()),
    StructField("aisle_id", IntegerType()),
    StructField("department_id", IntegerType())
])
struct_schema_aisle = StructType([
    StructField("aisle_id", IntegerType()),
    StructField("aisle", StringType())
])
struct_schema_depart = StructType([
    StructField("department_id", IntegerType()),
    StructField("department", StringType())
])


df_product = (
    spark.read.csv("/content/drive/MyDrive//products.csv", header=True, schema=struct_schema_product)
    .join(spark.read.csv("/content/drive/MyDrive/aisles.csv", header=True, schema=struct_schema_aisle), on="aisle_id", how="left")
    .join(spark.read.csv("/content/drive/MyDrive/departments.csv", header=True, schema=struct_schema_depart), on="department_id", how="left")
    .drop("aisle_id", "department_id")
)

In [ ]:
df = df.join(df_product, on="product_id", how="left")

In [ ]:
df.persist()

DataFrame[product_id: int, order_id: int, add_to_cart_order: smallint, reordered: smallint, product_name: string, aisle: string, department: string]

In [ ]:
%%time
df.show()

+----------+--------+-----------------+---------+--------------------+--------------------+------------+
|product_id|order_id|add_to_cart_order|reordered|        product_name|               aisle|  department|
+----------+--------+-----------------+---------+--------------------+--------------------+------------+
|     41890|      14|                6|        1|Organic Mesa Sunr...|              cereal|   breakfast|
|     26706|     976|               11|        1|Chicken Pad Thai ...|        frozen meals|      frozen|
|     19204|    1483|                5|        1|Organic Heritage ...|              cereal|   breakfast|
|     23015|    1561|                7|        1|Breaded Chicken N...|frozen appetizers...|      frozen|
|     15790|    1587|                3|        1|Organic Veggie Bites|frozen vegan vege...|      frozen|
|     28836|    2003|               15|        0|    Grapefruit Juice|        refrigerated|   beverages|
|     28836|    2458|               21|        1|    Gr

In [ ]:
df.rdd.getNumPartitions()

200

In [ ]:
from pyspark.sql.functions import spark_partition_id

df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0| 7560|
|          1|10425|
|          2| 8863|
|          3|37277|
|          4|12908|
|          5| 8603|
|          6|10918|
|          7| 4832|
|          8|13956|
|          9| 9574|
|         10|13446|
|         11|14256|
|         12|11716|
|         13|33766|
|         14|10167|
|         15| 8514|
|         16|17366|
|         17|10324|
|         18| 7266|
|         19|11594|
+-----------+-----+
only showing top 20 rows



In [ ]:
df = df.repartition("order_id")
df.persist()
df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|13408|
|          1|13026|
|          2|12404|
|          3|12657|
|          4|12859|
|          5|12954|
|          6|13052|
|          7|12900|
|          8|12429|
|          9|12682|
|         10|13118|
|         11|12480|
|         12|13166|
|         13|13540|
|         14|13176|
|         15|13155|
|         16|12998|
|         17|13573|
|         18|12676|
|         19|13053|
+-----------+-----+
only showing top 20 rows



In [ ]:
%%time

import pyspark.sql.functions as f
df_last_order_short = df.orderBy(["order_id", "add_to_cart_order"]).groupBy("order_id").agg(f.last("add_to_cart_order")).toPandas()

CPU times: user 2.26 s, sys: 175 ms, total: 2.43 s
Wall time: 39.7 s


In [ ]:
%%time

import pyspark.sql.functions as f
from pyspark.sql import Window

w = Window.partitionBy('order_id')
df_last_order = df.withColumn('max_add_to_cart_order', f.max('add_to_cart_order').over(w))\
    .where(f.col('add_to_cart_order') == f.col('max_add_to_cart_order'))\
    .drop('max_add_to_cart_order')
df_last_order_long = df_last_order.toPandas()

CPU times: user 2.43 s, sys: 235 ms, total: 2.66 s
Wall time: 14.8 s


In [ ]:
df_last_order_short.columns = ["order_id", "add_to_cart_order_short"]

In [ ]:
df_temp = df_last_order_long.merge(df_last_order_short, on="order_id")

In [ ]:
%%time
df_last_order.show()

+----------+--------+-----------------+---------+--------------------+--------------------+------------+
|product_id|order_id|add_to_cart_order|reordered|        product_name|               aisle|  department|
+----------+--------+-----------------+---------+--------------------+--------------------+------------+
|      1991|     833|                6|        1|Organic Lemonade ...|       juice nectars|   beverages|
|     22474|    6397|                7|        0|Cheddar Bunnies S...|            crackers|      snacks|
|     38274|    7554|               10|        0|Ice Cream Sandwic...|       ice cream ice|      frozen|
|     26348|    9465|                2|        0|Mixed Fruit Fruit...|fruit vegetable s...|      snacks|
|     30169|   10206|                4|        1|Total 2% All Natu...|              yogurt|  dairy eggs|
|     25194|   10362|               36|        1|Ground Sausage St...|tofu meat alterna...|        deli|
|     47209|   10817|                7|        0|Organi

In [ ]:
%%time
import pandas as pd

department_count = pd.concat([
    df_last_order.groupby("department").count().toPandas(),
    df.groupby("department").count().toPandas()
], axis=1)

CPU times: user 152 ms, sys: 17.8 ms, total: 170 ms
Wall time: 20.8 s


In [ ]:
department_count

,department,count,department,count
0,beverages,25659,beverages,215922
1,meat seafood,5709,meat seafood,56257
2,frozen,18980,frozen,180360
3,deli,8513,deli,83776
4,dry goods pasta,6662,dry goods pasta,70196
5,bulk,335,bulk,2749
6,other,440,other,3033
7,babies,2240,babies,34000
8,bakery,9078,bakery,94286
9,produce,66449,produce,760789


In [ ]:
%%time
df = df.orderBy(["order_id", "add_to_cart_order"])

df_temp = df.select(*df.columns, f.lag("aisle").over(w.partitionBy(f.lit(1)).orderBy("order_id")).alias("aisle_before"))
df_temp = df_temp.filter(f.col("aisle") != f.col("aisle_before"))
df_temp = df_temp.filter(f.col("add_to_cart_order") != 1).toPandas()

CPU times: user 17.4 s, sys: 3.03 s, total: 20.4 s
Wall time: 1min 29s
